In [1]:
import os
import pandas as pd
import numpy as np
import json
from urllib.request import urlopen
from bs4 import BeautifulSoup
# import progressbar
from tqdm import tqdm, tqdm_notebook

In [2]:
from selenium.webdriver import ChromeOptions
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options

# Let's do it for Wilayas

In [3]:
df_wilaya = pd.read_csv('wilayas_osm.csv')

In [4]:
df_wilaya.head()

,wilaya,OSM_num,Code,dairas_num,municipalities_num,area,population_2008,pop_density
0,Adrar,1258650,1,11,28,439700,399714,0.94
1,Chlef,1283740,2,13,35,4795,1002088,209.00
2,Laghouat,1280081,3,10,24,25057,455602,18.00
3,Oum El Bouaghi,1278749,4,12,29,7638,621612,81.00
4,Batna,1278748,5,21,61,12192,1119791,92.00


In [5]:
init_json = {
  "type": "FeatureCollection",
  "features": []
}

In [8]:
init_json['features']

[]

In [7]:
df_wilaya['wilaya'][0]

'Adrar'

In [9]:
df_wilaya['pop_density'][1]

209.0

In [10]:
for idx in tqdm(range(df_wilaya.shape[0])):

    OSM_number = df_wilaya['OSM_num'][idx]
    
    try:
        url_json = 'http://polygons.openstreetmap.fr/get_geojson.py?id=' + str(OSM_number) + '&params=0.004000-0.001000-0.001000'
        response = urlopen(url_json)
        wilaya_shape = json.load(response)

    except Exception:
        try:
            # we don't want to catch the IOError if it's raised
            url_json = 'http://polygons.openstreetmap.fr/get_geojson.py?id=' + str(OSM_number) + '&params=0.040000-0.010000-0.010000'
            with urlopen(url_json) as response:
                wilaya_shape = json.load(response)
        except:
            try:
                url_json = 'http://polygons.openstreetmap.fr/get_geojson.py?id=' + str(OSM_number) + '&params=0'
                with urlopen(url_json) as response:
                    wilaya_shape = json.load(response)
            except:
                print(OSM_number)
                print('second failed also')


    json_to_append = {
        "type": "Feature",
        "properties": {
            "NAME": df_wilaya['wilaya'][idx],
            "OMS_NUM": df_wilaya['OSM_num'][idx].tolist(),
            "CODE": df_wilaya['Code'][idx].tolist(),
            "MUNICIPALITIES_NUM": df_wilaya['dairas_num'][idx].tolist(),
            "AREA": df_wilaya['area'][idx].tolist(),
            "POPULATION_2008": df_wilaya['population_2008'][idx].tolist(),
            "POP_DENSITY": df_wilaya['pop_density'][idx].tolist()
            },
            "geometry": wilaya_shape
        }


    init_json['features'].append(json_to_append)    

100%|██████████| 58/58 [00:05<00:00,  9.77it/s]


In [11]:
init_json['features'][0]['properties']

{'NAME': 'Adrar',
 'OMS_NUM': 1258650,
 'CODE': 1,
 'MUNICIPALITIES_NUM': 11,
 'AREA': 439700,
 'POPULATION_2008': 399714,
 'POP_DENSITY': 0.94}

In [12]:
type(init_json['features'][0]['properties']['AREA'])

int

In [15]:
test=init_json['features'][0]['properties']['AREA']
type(test)

int

In [16]:
with open('algeria_wilaya.json', 'w') as outfile:
    json.dump(init_json, outfile)

## Let's do it for towns

In [44]:
df_cities = pd.read_csv('serial_number_cities.csv')

In [45]:
df_cities = df_cities.drop(columns=['Unnamed: 0'])

In [46]:
df_cities.columns

Index(['Phone number', 'Fax', 'Email', 'Website', 'Address', 'locality',
       'administrative_area_level_2', 'administrative_area_level_1', 'country',
       'relation_number'],
      dtype='object')

In [47]:
df_cities

,Phone number,Fax,Email,Website,Address,locality,administrative_area_level_2,administrative_area_level_1,country,relation_number
0,049 36 74 18,049 96 72 89,communeadrar@hotmail.com,communeadrar.dz,ADRAR,Adrar,Adrar,Adrar,Algeria,4171602
1,049.94.61.61,049.94.61.62,tamesttm@gmail.com,NaN,TAMEST,Tamest,Fenoughil,Adrar,Algeria,4171622
2,049.90.62.01,049.90.64.07,apccharouine@gmail.com,NaN,CHAROUINE,Charouine,Charouine,Adrar,Algeria,4171610
3,049.97.00.70,049.97.04.89,communereggane@gmail.com,NaN,REGGANE,Reggane,Reggane,Adrar,Algeria,4171617
4,049 97 24 97,049 97 23 87,commune.inzegmir.adrar@gmail.com,NaN,TIDMAIN,In Zghmir,Zaouiat Kounta,Adrar,Algeria,4171611
...,...,...,...,...,...,...,...,...,...,...
1530,046 85 31 10,046 85 31 26,NaN,www.apc.hadechkala.dz,COMMUNE HAD ECHKALLA,Had Echkalla,Ain Tarek,Relizane,Algeria,6559390
1531,046 74 92 33,046 74 92 32,bendaoud4835@hotmail.com,NaN,RUE DES FRERES AHSEN BENDAOUD,Bendaoud,Relizane,Relizane,Algeria,6559349
1532,046 84 91 18,046 84 91 28,elouldja4836@gmail.com,,Commune Ouldja Daira Ammi Moussa W Relizane,El Ouldja,Ammi Moussa,Relizane,Algeria,6559386
1533,046 84 61 69,046 84 61 68,apcmerj@gmail.com,NaN,Cité 05 juillet 1962,Merdja Sidi Abed,Oued Rhiou,Relizane,Algeria,6559408


In [54]:
df_cities = df_cities.fillna('No info')

In [55]:
cities_json = {
  "type": "FeatureCollection",
  "features": []
}

In [56]:

for idx in tqdm(range(df_cities.shape[0])):

    OSM_number = df_cities['relation_number'][idx]
    
    try:
        url_json = 'http://polygons.openstreetmap.fr/get_geojson.py?id=' + str(OSM_number) + '&params=0.004000-0.001000-0.001000'
        response = urlopen(url_json)
        city_shape = json.load(response)

    except Exception:
        try:
            # we don't want to catch the IOError if it's raised
            url_json = 'http://polygons.openstreetmap.fr/get_geojson.py?id=' + str(OSM_number) + '&params=0.040000-0.010000-0.010000'
            with urlopen(url_json) as response:
                city_shape = json.load(response)
        except:
            try:
                url_json = 'http://polygons.openstreetmap.fr/get_geojson.py?id=' + str(OSM_number) + '&params=0'
                with urlopen(url_json) as response:
                    city_shape = json.load(response)
            except:
                print(OSM_number)
                print('second failed also')


    json_to_append = {
        "type": "Feature",
        "properties": {
            'phone_number': df_cities['Phone number'][idx],
            'fax': df_cities['Fax'][idx],
            'email': df_cities['Email'][idx], 
            'website': df_cities['Website'][idx],
            'address': df_cities['Address'][idx], 
            'locality': df_cities['locality'][idx],
            'administrative_area_level_2': df_cities['administrative_area_level_2'][idx], 
            'administrative_area_level_1': df_cities['administrative_area_level_1'][idx], 
            'country': df_cities['country'][idx],
            'OSM_number': df_cities['relation_number'][idx].tolist()
            },
            "geometry": city_shape
        }


    cities_json['features'].append(json_to_append)    

100%|██████████| 1535/1535 [03:05<00:00,  8.26it/s]


In [57]:
test=cities_json['features'][0]['properties']
type(test)

dict

In [58]:
test

{'phone_number': '049 36 74 18',
 'fax': '049 96 72 89',
 'email': 'communeadrar@hotmail.com',
 'website': 'communeadrar.dz',
 'address': 'ADRAR',
 'locality': 'Adrar',
 'administrative_area_level_2': 'Adrar',
 'administrative_area_level_1': 'Adrar',
 'country': 'Algeria',
 'OSM_number': 4171602}

In [30]:
!conda install simplejson -y

Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda3

  added / updated specs:
    - simplejson


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    simplejson-3.17.0          |   py37h1de35cc_0          98 KB
    ------------------------------------------------------------
                                           Total:          98 KB

The following NEW packages will be INSTALLED:

  simplejson         pkgs/main/osx-64::simplejson-3.17.0-py37h1de35cc_0



simplejson-3.17.0    | 98 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [59]:
with open('algeria_cities.json', 'w') as outfile:
    json.dump(cities_json, outfile)

In [42]:
cities_json

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'phone_number': '049 36 74 18',
    'fax': '049 96 72 89',
    'email': 'communeadrar@hotmail.com',
    'website': 'communeadrar.dz',
    'address': 'ADRAR',
    'locality': 'Adrar',
    'administrative_area_level_2': 'Adrar',
    'administrative_area_level_1': 'Adrar',
    'country': 'Algeria',
    'OSM_number': 4171602},
   'geometry': {'type': 'MultiPolygon',
    'coordinates': [[[[-0.314, 27.987000000000002],
       [-0.316, 27.999000000000002],
       [-0.285, 28.065],
       [-0.049, 28.169],
       [-0.046, 28.169],
       [-0.044, 28.167],
       [-0.078, 28.009],
       [-0.073, 27.996000000000002],
       [-0.096, 27.828],
       [-0.146, 27.764],
       [-0.151, 27.764],
       [-0.153, 27.766000000000002],
       [-0.154, 27.772000000000002],
       [-0.181, 27.796],
       [-0.199, 27.809],
       [-0.218, 27.82],
       [-0.252, 27.847],
       [-0.264, 27.852],
       [-0.278, 27.851],
    

In [ ]:
options = ChromeOptions()
options.add_argument('--headless')
options.add_argument('--ignore-certificate-errors')
options.add_argument("--test-type")
browser = Chrome()
browser.get("http://polygons.openstreetmap.fr/index.py")

"""search the name of the city"""
search_polygon= browser.find_element_by_name("id")
search_pol ygon.send_keys(OSM_number)
submit_button=browser.find_element_by_xpath("//input[@type='submit']")
submit_button.click()